# Анализ зданий

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

df_full = pd.read_csv("data/IFC_2000_300_Columns_Selection.csv", low_memory=False)
df_full['Filename'] = df_full['Filename'].str.replace('_ifc', '').str.replace('_', ' ')
df_full.value_counts("Filename")

Filename
SantoryTest1                           173988
Prefab                                  30570
Uitbreiding stadion Heracles-Almelo     28132
12184-P Luchtwachttoren Bladel V2       26915
BIM model                               16132
Name: count, dtype: int64

In [43]:
df_full.groupby(['Filename', 'IfcEntity'])['IfcEntity'].count().unstack(fill_value=0)

IfcEntity,IfcBeam,IfcBuildingElementPart,IfcColumn,IfcCovering,IfcDiscreteAccessory,IfcFooting,IfcGrid,IfcMechanicalFastener,IfcMember,IfcPlate,IfcSlab,IfcWall
Filename,,,,,,,,,,,,
12184-P Luchtwachttoren Bladel V2,25631,0,598,0,0,4,0,549,0,8,88,37
BIM model,12121,0,45,3332,0,0,0,0,578,56,0,0
Prefab,16692,0,2751,1018,5635,0,0,96,184,0,3818,376
SantoryTest1,12482,0,6360,0,152234,0,56,0,2828,28,0,0
Uitbreiding stadion Heracles-Almelo,13927,900,4924,1765,5518,0,0,16,651,336,24,71


### Веса балок

In [32]:
df_full\
    .loc[lambda x: x['IfcEntity'] == 'IfcBeam']\
    .groupby('Filename')['NetWeight'].describe()

,count,mean,std,min,25%,50%,75%,max
Filename,,,,,,,,
12184-P Luchtwachttoren Bladel V2,25631.0,0.975060,3.364243,0.005118,0.019423,0.024302,0.024302,160.526250
BIM model,12121.0,25.808034,43.784575,0.000000,2.844212,2.844212,37.927967,369.589555
Prefab,16692.0,19.980557,317.741952,0.000000,0.299743,0.738921,1.476510,12330.525000
SantoryTest1,12482.0,61.422041,227.045421,1.145472,9.355126,20.081792,20.398539,3368.821927
Uitbreiding stadion Heracles-Almelo,13927.0,445.528068,1951.959584,0.000000,0.639451,3.972885,19.895040,33148.397876


### Суммарная площадь стен

In [58]:
df_full\
    .loc[lambda x: x['IfcEntity'] == 'IfcWall']\
    .groupby('Filename')['NetSurfaceArea'].sum()

Filename
12184-P Luchtwachttoren Bladel V2         92.051201
Prefab                                 13166.169810
Uitbreiding stadion Heracles-Almelo     2031.654715
Name: NetSurfaceArea, dtype: float64

### Мелкие детали `IfcDiscreteAccessory` по зданиям

In [19]:
df_full\
    .loc[lambda x: x['IfcEntity'] == 'IfcDiscreteAccessory']\
    .groupby('Filename')\
    .apply(lambda x: x['Name'].value_counts().head(5), include_groups=False)

Filename                             Name        
Prefab                               BetonSparing      2788
                                     vulpijp           1964
                                     betonsparing       862
                                     Plaat               14
                                     Koker                2
SantoryTest1                         PLATE           108288
                                     JOIST            27148
                                     STIFFENER         4344
                                     COLUMN            3868
                                     TURNBUCKLE        2872
Uitbreiding stadion Heracles-Almelo  GAIN              2879
                                     HALVE_BOL          760
                                     PLAAT              712
                                     BUIS               617
                                     LIGGER             214
Name: count, dtype: int64

### Топ-10 размеров панелей

In [47]:
df_full\
    .loc[lambda x: x['IfcEntity'] == 'IfcDiscreteAccessory']\
    .loc[lambda x: x['Name'] == 'PLATE']\
    .loc[:, ['Length', 'Width', 'ObjectType']]\
    .value_counts()\
    .head(10)

Length  Width  ObjectType
107.0   6.0    PL6*27        13632
320.0   9.0    PL9*80         9968
155.0   6.0    PL6*60         7200
               PL6*65         7068
320.0   9.0    PL9*200        5088
               PL9*150        5088
205.9   9.0    PL9*125        4880
155.0   6.0    PL6*80         3776
177.0   6.0    PL6*113        3408
256.0   16.0   PL16*96        3344
Name: count, dtype: int64